In [1]:
import driver
from pynq.pl_server.device import Device

import os
import my_metrics
import pandas as pd

# Accelerator

In [2]:
my_device = Device.devices[0]

accel = driver.FINNExampleOverlay(
    bitfile_name = '../bitfile/finn-accel.bit', 
    platform = "zynq-iodma",
    io_shape_dict = driver.io_shape_dict, 
    batch_size = 1,
    runtime_weight_dir = "runtime_weights/", 
    device=my_device
)

# Throughput Test

In [3]:
res = accel.throughput_test()
file = open("nw_metrics_eval.txt", "w")
file.write(str(res))
file.close()
print("Results written to nw_metrics_eval.txt")

Results written to nw_metrics_eval.txt


In [4]:
for key, value in res.items():
    print(f'{key}: {value}')

runtime[ms]: 2.345561981201172
throughput[images/s]: 426.3370603781256
DRAM_in_bandwidth[MB/s]: 64.17566502459849
DRAM_out_bandwidth[MB/s]: 0.0008526741207562512
fclk[mhz]: 100.0
batch_size: 1
fold_input[ms]: 0.13327598571777344
pack_input[ms]: 0.09655952453613281
copy_input_data_to_device[ms]: 2.3543834686279297
copy_output_data_from_device[ms]: 0.3452301025390625
unpack_output[ms]: 2.6178359985351562
unfold_output[ms]: 0.0476837158203125


# Images List and Labels Folder

### DFire

In [5]:
dfire_imgs_folder = '../../../../datasets/dfire_test/images/'
dfire_imgs_list = [dfire_imgs_folder + fname for fname in os.listdir(dfire_imgs_folder) if not fname.startswith('.')] 

dfire_labels_folder = '../../../../datasets/dfire_test/labels/'

In [6]:
print(f'DFire Test Dataset Length: {len(dfire_imgs_list)}')

DFire Test Dataset Length: 4306


### FASDD UAV

In [7]:
fasdd_uav_imgs_folder = '../../../../datasets/fasdd_uav_test/'
fasdd_uav_imgs_list = [fasdd_uav_imgs_folder + fname for fname in os.listdir(fasdd_uav_imgs_folder) if not fname.startswith('.')] 

In [8]:
print(f'FASDD UAV Test Dataset Length: {len(fasdd_uav_imgs_list)}')

FASDD UAV Test Dataset Length: 4181


### FASDD CV

In [9]:
fasdd_cv_imgs_folder = '../../../../datasets/fasdd_cv_test/'
fasdd_cv_imgs_list = [fasdd_cv_imgs_folder + fname for fname in os.listdir(fasdd_cv_imgs_folder) if not fname.startswith('.')] 

In [10]:
print(f'FASDD CV Test Dataset Length: {len(fasdd_cv_imgs_list)}')

FASDD CV Test Dataset Length: 15884


# Dataset Dictionary

In [20]:
dfire_dict = {"dfire": dfire_imgs_list}
fasdd_uav_dict = {"fasdd_uav": fasdd_uav_imgs_list}
fasdd_cv_dict = {"fasdd_cv": fasdd_cv_imgs_list}

In [12]:
full_ds_dict = {
    "dfire": dfire_imgs_list,
    "fasdd_uav": fasdd_uav_imgs_list,
    "fasdd_cv": fasdd_cv_imgs_list,
}

# Evaluate Full Dataset and Each Dataset Separately

### FULL Test Dataset

In [13]:
metrics_res = my_metrics.eval_accel(
    accel = accel,
    datasets_dict = full_ds_dict,
    dfire_labels_dir = dfire_labels_folder,
)

Number of test samples dfire: 4306



Testing: 100%|██████████| 4306/4306 [07:01<00:00, 10.21it/s]


Number of test samples fasdd_uav: 4181



Testing: 100%|██████████| 4181/4181 [31:54<00:00,  2.18it/s]


Number of test samples fasdd_cv: 15884



Testing: 100%|██████████| 15884/15884 [28:42<00:00,  9.22it/s] 


Smoke -> TP: 10763.0, FP: 522.0, TN: 12359.0, FN: 727.0
Fire  -> TP: 7759.0, FP: 482.0, TN: 15987.0, FN: 143.0


In [14]:
metrics_df = pd.DataFrame(metrics_res)
metrics_df

,Smoke,Fire,Mean
Accuracy,0.9488,0.9744,0.9616
Precision,0.9537,0.9415,0.9476
Recall,0.9367,0.9819,0.9593
F1,0.9452,0.9613,0.9532


### Only DFire Dataset

In [15]:
metrics_res = my_metrics.eval_accel(
    accel = accel,
    datasets_dict = dfire_dict,
    dfire_labels_dir = dfire_labels_folder,
)

Number of test samples dfire: 4306



Testing: 100%|██████████| 4306/4306 [06:59<00:00, 10.26it/s]

Smoke -> TP: 1895.0, FP: 152.0, TN: 2073.0, FN: 186.0
Fire  -> TP: 1053.0, FP: 62.0, TN: 3129.0, FN: 62.0


In [16]:
metrics_df = pd.DataFrame(metrics_res)
metrics_df

,Smoke,Fire,Mean
Accuracy,0.9215,0.9712,0.9464
Precision,0.9257,0.9444,0.9350
Recall,0.9106,0.9444,0.9275
F1,0.9181,0.9444,0.9312


### Only FASDD UAV

In [17]:
metrics_res = my_metrics.eval_accel(
    accel = accel,
    datasets_dict = fasdd_uav_dict,
)

Number of test samples fasdd_uav: 4181



Testing: 100%|██████████| 4181/4181 [31:58<00:00,  2.18it/s]

Smoke -> TP: 2034.0, FP: 7.0, TN: 2025.0, FN: 115.0
Fire  -> TP: 1320.0, FP: 36.0, TN: 2807.0, FN: 18.0


In [18]:
metrics_df = pd.DataFrame(metrics_res)
metrics_df

,Smoke,Fire,Mean
Accuracy,0.9708,0.9871,0.9790
Precision,0.9966,0.9735,0.9850
Recall,0.9465,0.9865,0.9665
F1,0.9709,0.9800,0.9754


### Only FASDD CV

In [21]:
metrics_res = my_metrics.eval_accel(
    accel = accel,
    datasets_dict = fasdd_cv_dict,
)

Number of test samples fasdd_cv: 15884



Testing: 100%|██████████| 15884/15884 [28:49<00:00,  9.18it/s] 


Smoke -> TP: 6834.0, FP: 363.0, TN: 8261.0, FN: 426.0
Fire  -> TP: 5386.0, FP: 384.0, TN: 10051.0, FN: 63.0


In [22]:
metrics_df = pd.DataFrame(metrics_res)
metrics_df

,Smoke,Fire,Mean
Accuracy,0.9503,0.9719,0.9611
Precision,0.9496,0.9334,0.9415
Recall,0.9413,0.9884,0.9648
F1,0.9454,0.9602,0.9528
